In [ ]:
%%bash
pip install pyspark

if [[ ! -f ./train.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/health-insurance-cross-sell-prediction/main/data/train.csv
fi

if [[ ! -f ./test.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/health-insurance-cross-sell-prediction/main/data/test.csv  
fi

  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=800f1c4bccbe3cb0c89b99d538d6713352d3bff74e44c7e61b87760b6e8ba2af
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


--2020-11-15 01:15:17--  https://raw.githubusercontent.com/aatishsuman/health-insurance-cross-sell-prediction/main/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21432357 (20M) [text/plain]
Saving to: ‘train.csv’

     0K .......... .......... .......... .......... ..........  0% 5.81M 4s
    50K .......... .......... .......... .......... ..........  0% 6.90M 3s
   100K .......... .......... .......... .......... ..........  0% 23.0M 2s
   150K .......... .......... .......... .......... ..........  0% 27.8M 2s
   200K .......... .......... .......... .......... ..........  1% 9.08M 2s
   250K .......... .......... .......... .......... ..........  1% 66.0M 2s
   300K .......... .......... .......... .......... ..........  1% 46.1M 2s
   350K ...

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.ml import feature, Pipeline, regression, classification, evaluation, tuning
import numpy as np
import pandas as pd

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
train = spark.read.csv('train.csv', inferSchema=True, header=True)
test = spark.read.csv('test.csv', inferSchema=True, header=True)

print(train.toPandas().shape, test.toPandas().shape)

(381109, 12) (127037, 11)


In [ ]:
train.toPandas().head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [ ]:
numerical_columns=['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
categorical_columns=['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Response']

In [ ]:
feature_engineering_pipe = Pipeline(stages=[feature.StringIndexerModel.from_labels(['Male', 'Female'], inputCol='Gender', outputCol='Gender_Feature'), 
                                            feature.StringIndexerModel.from_labels(['< 1 Year', '1-2 Year', '> 2 Years'], inputCol='Vehicle_Age', outputCol='Vehicle_Age_Feature'), 
                                            feature.StringIndexerModel.from_labels(['No', 'Yes'], inputCol='Vehicle_Damage', outputCol='Vehicle_Damage_Feature')])
train_xformed = feature_engineering_pipe.fit(train).transform(train).select(numerical_columns + 
                                                                            ['Driving_License', 'Previously_Insured', 'Response', 
                                                                             fn.col('Gender_Feature').alias('Gender'), 
                                                                             fn.col('Vehicle_Age_Feature').alias('Vehicle_Age'), 
                                                                             fn.col('Vehicle_Damage_Feature').alias('Vehicle_Damage')])

In [ ]:
train_xformed.toPandas().head()

,Age,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage,Driving_License,Previously_Insured,Response,Gender,Vehicle_Age,Vehicle_Damage
0,44,28.0,40454.0,26.0,217,1,0,1,0.0,2.0,1.0
1,76,3.0,33536.0,26.0,183,1,0,0,0.0,1.0,0.0
2,47,28.0,38294.0,26.0,27,1,0,1,0.0,2.0,1.0
3,21,11.0,28619.0,152.0,203,1,1,0,0.0,0.0,0.0
4,29,41.0,27496.0,152.0,39,1,1,0,1.0,0.0,0.0


In [ ]:
def get_mse(features):
  train_df, validation_df = train_xformed.randomSplit([0.9, 0.1], 42)
  pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=features, outputCol='features'), regression.LinearRegression(labelCol='Response')])
  evaluator = evaluation.RegressionEvaluator(labelCol='Response', metricName='mse')
  model = pipe.fit(train_df)
  return evaluator.evaluate(model.transform(validation_df)), dict(zip(model.stages[-2].getInputCols(), model.stages[-1].summary.pValues))

def get_stepwise_pred_list():
  predictors = numerical_columns + categorical_columns[:-1]
  while (len(predictors) > 1):
    initial_mse, p_values = get_mse(predictors) 
    predictors = list(sorted(p_values, key=p_values.get, reverse=True))
    predictors.pop(0)
    mse, p_values = get_mse(predictors)
    predictors = list(sorted(p_values, key=p_values.get, reverse=True))
    if (mse >= initial_mse):
      return predictors[::-1]
  return predictors[::-1]

best_predictors = get_stepwise_pred_list()
best_predictors

['Vehicle_Damage',
 'Vehicle_Age',
 'Previously_Insured',
 'Driving_License',
 'Policy_Sales_Channel',
 'Annual_Premium',
 'Age']

In [ ]:
# getting baseline
train_xformed.groupBy('Response').count().withColumn('class percentage', fn.col('count') / train_xformed.count()).show()

+--------+------+-------------------+
|Response| count|   class percentage|
+--------+------+-------------------+
|       1| 46710|0.12256336113815208|
|       0|334399|  0.877436638861848|
+--------+------+-------------------+



In [ ]:
# using best predictors
nb_train = train_xformed.withColumnRenamed('Response', 'label')
train_df, validation_df = nb_train.randomSplit([0.9, 0.1], 42)
nb_pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=best_predictors, outputCol='features'), classification.NaiveBayes()])
nb_model = nb_pipe.fit(train_df)
train_predictions = nb_model.transform(train_df)
evaluator = evaluation.BinaryClassificationEvaluator(metricName='areaUnderPR')
validation_predictions = nb_model.transform(validation_df)
print('Validation PR AUC: {}'.format(evaluator.evaluate(validation_predictions)))

accuracy = validation_predictions.where('label == prediction').count() / validation_df.count()
print('Validation accuracy {}'.format(accuracy))

Validation PR AUC: 0.16019855997204668
Validation accuracy 0.6809346621217246


In [ ]:
%%time
nb = classification.NaiveBayes()
nb_pipe_cross_val = Pipeline(stages=[feature.VectorAssembler(inputCols=best_predictors, outputCol='features'), nb])
paramGrid = tuning.ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.5, 2.0]).build()
crossval = tuning.CrossValidator(estimator=nb_pipe_cross_val, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=2)
nb_model_cross_val = crossval.fit(train_df)
print('Validation PR AUC: {}'.format(evaluator.evaluate(nb_model_cross_val.transform(validation_df))))
print('Best smoothing value: {}'.format(nb_model_cross_val.bestModel.stages[-1].getSmoothing()))

Validation PR AUC: 0.16020098876340347
Best smoothing value: 0.0
CPU times: user 733 ms, sys: 155 ms, total: 887 ms
Wall time: 1min 28s


In [ ]:
# using best predictors + Gender
nb_pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=best_predictors + ['Gender'], outputCol='features'), classification.NaiveBayes()])
nb_model = nb_pipe.fit(train_df)
train_predictions = nb_model.transform(train_df)
validation_predictions = nb_model.transform(validation_df)
print('Validation PR AUC: {}'.format(evaluator.evaluate(validation_predictions)))

accuracy = validation_predictions.where('label == prediction').count() / validation_df.count()
print('Validation accuracy {}'.format(accuracy))

Validation PR AUC: 0.1607042449470127
Validation accuracy 0.6807264968384897
